In [1]:
import pandas
import numpy as np
import time
import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
#  функция для снятия таймера
def log_timer(fn, args):
    start_time = datetime.datetime.now()
    result = fn(*args)
    time_elapsed = datetime.datetime.now() - start_time;
    print('Function {} with args: {} executed: {}'.format(fn.__name__, args[0], time_elapsed))
    return result

print_result = lambda param: print('Param: {:8} score: {:.4}'.format(param[0], param[1])) 

In [3]:
# Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 

features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [4]:
# Удалите признаки, связанные с итогами матча 
# (они помечены в описании данных как отсутствующие в тестовой выборке).
data = features[features.columns[:-6]]

In [5]:
# Проверьте выборку на наличие пропусков с помощью функции count(), 
# которая для каждого столбца показывает число заполненных значений. 
# Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, 
# и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.
count_by_col = data.isnull().sum()
count_by_col[count_by_col > 0]

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

first_blood_time - игровое время первой крови
first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
first_blood_player1: игрок, причастный к событию
first_blood_player2: второй игрок, причастный к событию

В 19553 играх из 97230 первая кровь не была пролита. Соответственно first_blood_team, first_blood_player1 должны быть пустыми 19553, так как ни одна команда и ни один игрок не совершили первую кровь. 
В колонке first_blood_player2 пропусков больше - 43987, это означает что в 24434 (43987-19553) случаях первую кровь совершил один игрок. 

In [6]:
# Замените пропуски на нули с помощью функции fillna(). 
# На самом деле этот способ является предпочтительным для логистической регрессии, 
# поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. 
# Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое 
# или очень маленькое значение — в этом случае при построении разбиения вершины 
# можно будет отправить объекты с пропусками в отдельную ветвь дерева. 
# Также есть и другие подходы — например, замена пропуска на среднее значение признака. 
# Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков 
# и сравните их между собой.
X = data.fillna(0)

In [7]:
# Какой столбец содержит целевую переменную? Запишите его название.
result_column = 'radiant_win'
y = features[result_column]

## Подход 1: градиентный бустинг "в лоб"

In [8]:
# Забудем, что в выборке есть категориальные признаки, 
# и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". 
# Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), 
# не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени,
# и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. 

kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
def GBC(n_estimators, X, y):
    clf = GradientBoostingClassifier(n_estimators=n_estimators, random_state=42)
    score = cross_val_score(clf, X, y, cv=kf, scoring='roc_auc').mean()
    return score

In [9]:
# Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, 
# попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения 
# для количества деревьев: 10, 20, 30). 
number_trees = [10, 20, 30, 40, 100]

scores = {n: log_timer(GBC, (n, X, y)) for n in number_trees}

Function GBC with args: 10 executed: 0:00:25.351497
Function GBC with args: 20 executed: 0:00:49.528454
Function GBC with args: 30 executed: 0:01:13.603098
Function GBC with args: 40 executed: 0:01:38.505149
Function GBC with args: 100 executed: 0:04:07.328606


<b>Долго ли настраивались классификаторы?</b>

Чем больше деревьев используется, тем дольше время простчета. 

In [10]:
print('ALL_RESULT:')
list(map(print_result, scores.items()))

max_result = max([x for x in scores.values()])
print('Best result: {:.4}'.format(max_result))

ALL_RESULT:
Param:       10 score: 0.6649
Param:       20 score: 0.6825
Param:       30 score: 0.69
Param:       40 score: 0.694
Param:      100 score: 0.7062
Best result: 0.7062


<b>Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество,
скорее всего, продолжит расти при дальнейшем его увеличении?</b>

Качество скорее всего продолжит расти, так же как и время просчета.

## Подход 2: логистическая регрессия

In [11]:
# Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! 
# Может пригодиться sklearn.preprocessing.StandartScaler.

def scaler_data(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X)

In [12]:
# Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) 
# с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. 
def LG(C, X, y):
    kf = KFold(n_splits=5, shuffle=True)
    lg = LogisticRegression(C=C, random_state=42, max_iter=3000)
    score = cross_val_score(lg, X, y, cv=kf, scoring='roc_auc').mean()
    return score

In [13]:
# Подберите при этом лучший параметр регуляризации (C). 
C_pow_range = range(-5, 6)
C_range = [10.0 ** i for i in C_pow_range]

In [14]:
X_scaler = scaler_data(X)
scores = {c: log_timer(LG, (c, X_scaler, y)) for c in C_range}

Function LG with args: 1e-05 executed: 0:00:00.901671
Function LG with args: 0.0001 executed: 0:00:01.072035
Function LG with args: 0.001 executed: 0:00:01.949980
Function LG with args: 0.01 executed: 0:00:03.047710
Function LG with args: 0.1 executed: 0:00:03.093510
Function LG with args: 1.0 executed: 0:00:03.247837
Function LG with args: 10.0 executed: 0:00:03.010252
Function LG with args: 100.0 executed: 0:00:03.210146
Function LG with args: 1000.0 executed: 0:00:03.138587
Function LG with args: 10000.0 executed: 0:00:02.902745
Function LG with args: 100000.0 executed: 0:00:03.321926


In [15]:
print('ALL_RESULT:')
list(map(print_result, scores.items()))

# Какое наилучшее качество у вас получилось? 
max_result = max([x for x in scores.values()])
print('Best result: {:.4}'.format(max_result))

ALL_RESULT:
Param:    1e-05 score: 0.6952
Param:   0.0001 score: 0.7114
Param:    0.001 score: 0.7163
Param:     0.01 score: 0.7163
Param:      0.1 score: 0.7166
Param:      1.0 score: 0.7163
Param:     10.0 score: 0.7164
Param:    100.0 score: 0.7166
Param:   1000.0 score: 0.7164
Param:  10000.0 score: 0.7164
Param: 100000.0 score: 0.7166
Best result: 0.7166


<b>Как качество соотносится с качеством градиентного бустинга?</b>

Качесвтво логистической регрессии выше чем у градиентного бустинга.

<b>Чем вы можете объяснить эту разницу? </b>

Вероятно, результат игры зависит от множества параметров, алгоритм градиентного бустинга не мложет учесть их все.

<b>Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?</b>

Да, логистическая регрессия работает значительно быстрее градиентного бустинга.

In [16]:
# Среди признаков в выборке есть категориальные, которые мы использовали как числовые, 
# что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: 
# lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. 
# Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке 
# с подбором лучшего параметра регуляризации. 
def delete_columns(X):
    return X.loc[:, ~X.columns.isin([
        'lobby_type', 
        'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero',
        ])]
X_digit = delete_columns(X)
X_digit_scaler = scaler_data(X_digit)

In [17]:
scores = {c: log_timer(LG, (c, X_digit_scaler, y)) for c in C_range}

Function LG with args: 1e-05 executed: 0:00:01.317453
Function LG with args: 0.0001 executed: 0:00:01.106577
Function LG with args: 0.001 executed: 0:00:02.071473
Function LG with args: 0.01 executed: 0:00:03.236805
Function LG with args: 0.1 executed: 0:00:03.182140
Function LG with args: 1.0 executed: 0:00:03.333552
Function LG with args: 10.0 executed: 0:00:03.610511
Function LG with args: 100.0 executed: 0:00:03.043978
Function LG with args: 1000.0 executed: 0:00:03.295592
Function LG with args: 10000.0 executed: 0:00:03.387650
Function LG with args: 100000.0 executed: 0:00:02.878598


In [18]:
print('ALL_RESULT:')
list(map(print_result, scores.items()))

max_result = max([x for x in scores.values()])
print('Best result: {:.4}'.format(max_result))

ALL_RESULT:
Param:    1e-05 score: 0.6951
Param:   0.0001 score: 0.7113
Param:    0.001 score: 0.716
Param:     0.01 score: 0.7165
Param:      0.1 score: 0.7164
Param:      1.0 score: 0.7165
Param:     10.0 score: 0.7164
Param:    100.0 score: 0.7163
Param:   1000.0 score: 0.7163
Param:  10000.0 score: 0.7165
Param: 100000.0 score: 0.7166
Best result: 0.7166


<b>Изменилось ли качество? </b>
Нет качество не изменилось.

<b>Чем вы можете это объяснить?</b>
Тем что эти данные не имели никакого влияния на результат.

In [19]:
# На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, 
# какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, 
# и некоторые из них выигрывают чаще, чем другие. Выясните из данных, 
# сколько различных идентификаторов героев существует в данной игре 
# (вам может пригодиться фукнция unique или value_counts).

heroes = pandas.read_csv('data/dictionaries/heroes.csv')
N = len(heroes) # — количество различных героев в выборке
print('Всего героев в игре:', len(heroes))

Всего героев в игре: 112


In [20]:
# Воспользуемся подходом "мешок слов" для кодирования информации о героях. 
# Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, 
# если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; 
# минус единице, если i-й герой играл за команду Dire. 
# Ниже вы можете найти код, который выполняет данной преобразование. 
def get_X_pick(X):
    X_pick = np.zeros((X.shape[0], N))
    for i, match_id in enumerate(X.index):
        for p in range(5):
            X_pick[i, X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
    return X_pick

X_pick = get_X_pick(X)

In [21]:
# Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
X_digit_scaler_with_word = np.hstack([X_digit_scaler,X_pick])
X_digit_scaler_with_word.shape

(97230, 203)

In [22]:
# Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
scores = {c: log_timer(LG, (c, X_digit_scaler_with_word, y)) for c in C_range}

Function LG with args: 1e-05 executed: 0:00:01.478195
Function LG with args: 0.0001 executed: 0:00:04.306259
Function LG with args: 0.001 executed: 0:00:05.093580
Function LG with args: 0.01 executed: 0:00:11.591283
Function LG with args: 0.1 executed: 0:00:15.620991
Function LG with args: 1.0 executed: 0:00:15.685798
Function LG with args: 10.0 executed: 0:00:14.562985
Function LG with args: 100.0 executed: 0:00:14.133328
Function LG with args: 1000.0 executed: 0:00:15.368152
Function LG with args: 10000.0 executed: 0:00:14.392651
Function LG with args: 100000.0 executed: 0:00:13.823294


In [23]:
print('ALL_RESULT:')
list(map(print_result, scores.items()))

# Какое получилось качество? 
max_result = max([x for x in scores.values()])
print('Best result: {:.4}'.format(max_result))

ALL_RESULT:
Param:    1e-05 score: 0.6992
Param:   0.0001 score: 0.7249
Param:    0.001 score: 0.7463
Param:     0.01 score: 0.7517
Param:      0.1 score: 0.7517
Param:      1.0 score: 0.7517
Param:     10.0 score: 0.7519
Param:    100.0 score: 0.7519
Param:   1000.0 score: 0.7518
Param:  10000.0 score: 0.7519
Param: 100000.0 score: 0.7519
Best result: 0.7519


<b>Улучшилось ли оно? </b>
Да качество возросло

<b>Чем вы можете это объяснить?</b>
Наличие определенных типов героев в игре имеет значение в определении результата игры.

In [24]:
# Постройте предсказания вероятностей победы команды Radiant для тестовой выборки 
# с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации).
# Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1],
# не совпадают между собой (т.е. что модель не получилась константной).
test = pandas.read_csv('./data/features_test.csv', index_col='match_id')
test.shape

(17177, 102)

In [25]:
X_test = test.fillna(0)

X_test_pick = get_X_pick(X_test) # мешок слов для тестовой выборки
X_test_digit = delete_columns(X_test) # удаляем колонки с типом героев
X_test_digit_scaler = scaler_data(X_test_digit) # скалируем данные
X_test_digit_scaler_with_word = np.hstack([X_digit_scaler,X_pick]) # объедтиняем с мешком слов

clf = LogisticRegression(C=1.0, random_state=42, max_iter=3000).fit(X_digit_scaler_with_word, y)
prediction = clf.predict_proba(X_test_digit_scaler_with_word)
prediction

array([[0.32894292, 0.67105708],
       [0.26343979, 0.73656021],
       [0.39695896, 0.60304104],
       ...,
       [0.53995305, 0.46004695],
       [0.58381955, 0.41618045],
       [0.06559848, 0.93440152]])

In [26]:
max([x[1] for x in prediction])

0.9987304098042353

In [27]:
min([x[1] for x in prediction])

0.0013221239361494441